#Imports

In [ ]:
#@title
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
!git clone https://github.com/bosemessi/StatsbombOpenData --q
!pip install tqdm --q
from tqdm import tqdm
import requests 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet('/content/StatsbombOpenData/WC2018.parquet').reset_index(drop=True)

#Calculate XGChain and XGBuildup from the sequence data

In [ ]:
#@title

def player_xgc(match_id):
    gamedf = df[(df.match_id==match_id)&(df.period<=4)].reset_index(drop=True)
    typemask = gamedf.type_name == 'Shot'
    openplay = gamedf.shot_type_name == 'Open Play'
    sameteam = gamedf.team_name == gamedf.possession_team_name
    gamedf['OPS'] = np.where(typemask & openplay & sameteam,1,0)
    gamedf['oneminusxG'] = 1.0 - gamedf['shot_statsbomb_xg']
    aggdict = {'OPS':'sum','oneminusxG':np.prod}
    grouped = gamedf[gamedf.OPS==1].groupby(['team_name','possession']).agg(aggdict).reset_index()
    grouped['oneminusxG'] = 1.0 - grouped['oneminusxG']
    grouped.rename(columns={'oneminusxG':'xGCond'},inplace=True)
    grouped.drop(columns='OPS',inplace=True)
    gamedf = gamedf.merge(grouped,how='left')
    gamedf['xGCond'].fillna(value=0,inplace=True)
    gamedf['xGCond'] = np.where(gamedf.type_name.isin(['Pass','Carry']),gamedf.xGCond,0)
    groupdf = gamedf.groupby(['player_name','possession']).agg({'xGCond':'mean'}).reset_index()
    return groupdf

def player_xgb(match_id):
    gamedf = df[(df.match_id==match_id)&(df.period<=4)].reset_index(drop=True)
    typemask = gamedf.type_name == 'Shot'
    openplay = gamedf.shot_type_name == 'Open Play'
    sameteam = gamedf.team_name == gamedf.possession_team_name
    gamedf['OPS'] = np.where(typemask & openplay & sameteam,1,0)
    gamedf['oneminusxG'] = 1.0 - gamedf['shot_statsbomb_xg']
    aggdict = {'OPS':'sum','oneminusxG':np.prod}
    grouped = gamedf[gamedf.OPS==1].groupby(['team_name','possession']).agg(aggdict).reset_index()
    grouped['oneminusxG'] = 1.0 - grouped['oneminusxG']
    grouped.rename(columns={'oneminusxG':'xGCond'},inplace=True)
    grouped.drop(columns='OPS',inplace=True)
    gamedf = gamedf.merge(grouped,how='left')
    gamedf['xGCond'].fillna(value=0,inplace=True)
    gamedf['xGCond'] = np.where(gamedf.type_name.isin(['Pass','Carry']),gamedf.xGCond,0)
    gamedf.loc[(gamedf.pass_shot_assist==True)|(gamedf.pass_goal_assist==True),
               'xGCond'] = 0
    groupdf = gamedf.groupby(['player_name','possession']).agg({'xGCond':'mean'}).reset_index()
    return groupdf

xgcdfs = []
xgbdfs = []
for g in tqdm(df.match_id.unique(),desc='Reading Games'):
    xgcdfs.append(player_xgc(g))
    xgbdfs.append(player_xgb(g))
xgcdf = pd.concat(xgcdfs, ignore_index=True)
xgbdf = pd.concat(xgbdfs, ignore_index=True)

Reading Games: 100%|██████████| 64/64 [00:26<00:00,  2.39it/s]


In [ ]:
xgbdf.rename(columns={'xGCond':'xGBuildup'},inplace=True)
xgcdf.rename(columns={'xGCond':'xGChain'},inplace=True)

g1 = xgbdf.groupby('player_name').xGBuildup.sum().reset_index()
g2 = xgcdf.groupby('player_name').xGChain.sum().reset_index()
len(g1), len(g2)

(603, 603)

In [ ]:
g1 = g1.merge(g2, how='left')
len(g1)

603

#Combine XGC and XGB and save

In [ ]:
g1

,player_name,xGBuildup,xGChain
0,Aaron Mooy,0.585616,1.158689
1,Abdalla Mahmoud El Said Bekhit,0.677549,0.824822
2,Abdiel Arroyo Molinar,0.063533,0.239129
3,Abdullah Ibrahim Al Maiouf,0.030863,0.030863
4,Abdullah Ibrahim Otayf,0.316491,0.340030
...,...,...,...
598,İlkay Gündoğan,0.330735,0.356063
599,Łukasz Fabiański,0.035246,0.035246
600,Łukasz Piszczek,0.427537,0.595599
601,Łukasz Teodorczyk,0.058117,0.058117


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
g1.to_parquet('/content/drive/MyDrive/Norwich/xGBC.parquet',index=False)